# Task 1: Binary Labeling of JEL Codes
***

We prep for fitting our model by labeling entries with codes and remove digits in the JEL Codes.

In [51]:
import re
import glob
import string

import pandas as pd
import numpy as np

In [82]:
#Get a dataframe of all csv's combined
counter = 0
df = None
for filename in glob.glob('csv_files_grid/*.csv'):
    if counter == 0:
        df = pd.read_csv(filename, header=None, low_memory=False)
        counter += 1
    elif counter < 500:
        new_df = pd.read_csv(filename, header=None, low_memory=False)
        new_df = new_df.drop([0])
        df = pd.concat([df, new_df])
        counter += 1
    else:
        break
        
df.to_csv('dataframe_pickles/original_frame_500.csv', encoding='utf-8-sig')      
df

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,Title,Abstract,Keywords,JEL Codes,Author1,Author2,Author3,Author4,Author5,Author6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,,"[ , corporate social responsibility, , corpor...",NaN,Дохолян Самвел Бахшиевич,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,,"[ , , , , financial stability, a balanced m...",NaN,Могзоев Александр Мушкудаевич,Шадрина Ирина Николаевна,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,,"[ , , , small business, state support, support]",NaN,Руденко Людмила Геннадьевна,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,,"[ , , , , , , , , ]",NaN,Суптело Наталья Петровна,Михайлов Павел Сергеевич,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,,,"[ , , , , , problems, statistical methodol...",NaN,Тебекин Алексей Васильевич,Тебекин Павел Алексеевич,Кузнецова Галина Васильевна,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,,,"[ , , , , , , capital repairs, an apartme...",NaN,Клюев Виктор Дмитриевич,Зайцев Дмитрий Анатольевич,Евсикова Юлия Владимировна,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,,35,"[ , , , , , communications, intraorganizat...",NaN,Алексеев Александр Николаевич,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,,,"[ , , urban agglomeration, municipal union]",NaN,Богославец Даниил Михайлович,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,,,"[ , , , , city, transport infrastructure, t...",NaN,Зубец Антон Желькович,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
df_jel = df[[0,1,3]]
df_jel = df_jel.dropna()
df_jel

,0,1,3
0,Title,Abstract,JEL Codes
80,,2 2,THE CORRECTION FACTOR K2
1,political hegemony or cultural hegemony,every political system including the democrati...,B30 Y80
2,nomos as decision carl schmitt,the territorial demarcation by which it is pos...,B30 Y80
3,absolute evil reflections on the political re...,recently gianfranco fini president of the ital...,B30 Y80
4,the history of new right ideas in australia,australians are currently discussing and debat...,B30 Y80
5,why music a look at art as propaganda,music can influence our emotions and the quali...,Y80
6,multiculturalism as a process of globalisation,multi cultural politics including that concern...,B30 Y80
7,the eurasian idea,different terms lose their original meaning th...,B30 Y80
8,positive christianity and the russian campaign...,there is a lot of evidence concerning hitler s...,Z12


In [120]:
#label jel codes
#find root codes
arr = df_jel[3].values[1:]

found = False
labels = ['O']
root_codes = ['Root Codes']

tmp = []
for a in arr:
    for b in a.split():
        for c in b:
            if c.isdigit() == False and c not in string.punctuation:
                tmp.append(c.upper())
                if c == 'O':
                    labels.append(1)
                    found = True
                    break
        
        if found == True:
            break
            
    root_codes.append(list(set(tmp)))
    tmp =[]
    if found == False:
        labels.append(0)
        
    found = False
    
df_jel.drop(3, axis=1, inplace=True)

In [121]:
df_jel[4] = df_jel[0] + ' ' + df_jel[1]
df_jel[5] = np.array(root_codes, dtype=object)
df_jel[6] = np.array(labels)
df_jel.drop([0,1], axis=1, inplace=True)
df_jel.drop(0, axis=0, inplace=True)
df_jel

,4,5,6
80,2 2,"[O, H, E, T, C]",1
1,political hegemony or cultural hegemony every...,"[Y, B]",0
2,nomos as decision carl schmitt the territorial...,"[Y, B]",0
3,absolute evil reflections on the political re...,"[Y, B]",0
4,the history of new right ideas in australia au...,"[Y, B]",0
5,why music a look at art as propaganda music ca...,[Y],0
6,multiculturalism as a process of globalisation...,"[Y, B]",0
7,the eurasian idea different terms lose their o...,"[Y, B]",0
8,positive christianity and the russian campaign...,[Z],0
67,earnings management surrounding ceo turnover e...,"[M, G]",0


In [122]:
df_jel.to_csv('dataframe_pickles/o_codes_extended_500.csv', encoding='utf-8-sig')